<h1>Data extraction from the movie database api</h1>

In [161]:
import numpy as np
import pandas as pd
import requests

In [2]:
#Configuration

KEY = 'c8706c769cd0fbbc13971c769cf73e76'

In [163]:
#Load Netflix data
mv_ratings = pd.read_csv('data/movies_subset_balanced.csv.gz', compression = 'gzip')
mv_info = pd.read_csv('data/movie_titles.csv', error_bad_lines=False, header=None, names = ['MovieID','YearOfRelease','Title'], encoding='latin1')

In [164]:
#Convert years into integers
mv_info = mv_info.dropna()
mv_info['YearOfRelease'] = mv_info['YearOfRelease'].astype(int)
mv_info.head()

,MovieID,YearOfRelease,Title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [165]:
mv_ratings.head()

,MovieID,UserID,Rating,RatingDate
0,30,1366162,1,2005-10-26
1,30,483101,1,2005-07-07
2,30,2148796,1,2004-11-20
3,30,998271,1,2004-06-21
4,30,607621,1,2005-06-07


In [166]:
nflix_db = mv_ratings.groupby('MovieID').count().reset_index()[['MovieID']]
nflix_db  = nflix_db.merge(mv_info, left_on = 'MovieID', right_on = 'MovieID').rename(columns = {'MovieID': 'nflix_id', 
                                                                                         'YearOfRelease': 'nflix_release_year',
                                                                                         'Title' : 'nflix_title'})
nflix_db.head()

,nflix_id,nflix_release_year,nflix_title
0,30,2003,Something's Gotta Give
1,175,1992,Reservoir Dogs
2,191,2003,X2: X-Men United
3,313,2000,Pay It Forward
4,457,2004,Kill Bill: Vol. 2


In [218]:
nflix_db[nflix_db['nflix_title']=='Radio']

,nflix_id,nflix_release_year,nflix_title
109,11040,2003,Radio


In [167]:
pd.set_option('display.max_rows', None)


<h2> Movie Database API</h2>

In [182]:
#Extract data from movie database api

def extract_data_movie_database_api_query(movie_titles):
    '''
    Query movie database with a movie title
    
    Input:
    movie_titles : list of movie titles
    
    Output:
    response_data : Dataframe which holds data on the query
    '''

    for i, title in enumerate(movie_titles):
        url = 'https://api.themoviedb.org/3/search/movie?'
        apikey = f'api_key={KEY}'
        language = '&language=en-US'
        page = '&page=1'
        adult = '&include_adult=false'
        query = f"&query={title}"
        response = requests.get(f'{url}{apikey}{language}{page}{adult}{query}')
        response_res = response.json()['results']
        if len(response_res) !=0:
            tmp = pd.DataFrame(response_res[0])
            if i == 0: 
                response_data = tmp
            else:
                response_data  = pd.concat([response_data, tmp])
    return response_data



In [183]:
#Extracted data from Movie Database Api
mvdb_db = extract_data_movie_database_api_query(nflix_db['nflix_title'])
mvdb_db.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/bgOoq2T5ABiLk1Zo9NBjWZdA1vM.jpg,18,6964,en,Something's Gotta Give,Harry Sanborn is an aged music industry exec w...,11.395,/4fQJjwbDf1KELvJJMcmAV6AxJdH.jpg,2003-12-12,Something's Gotta Give,False,6.6,1134
1,False,/bgOoq2T5ABiLk1Zo9NBjWZdA1vM.jpg,35,6964,en,Something's Gotta Give,Harry Sanborn is an aged music industry exec w...,11.395,/4fQJjwbDf1KELvJJMcmAV6AxJdH.jpg,2003-12-12,Something's Gotta Give,False,6.6,1134
2,False,/bgOoq2T5ABiLk1Zo9NBjWZdA1vM.jpg,10749,6964,en,Something's Gotta Give,Harry Sanborn is an aged music industry exec w...,11.395,/4fQJjwbDf1KELvJJMcmAV6AxJdH.jpg,2003-12-12,Something's Gotta Give,False,6.6,1134
0,False,/g6R1OT7ETBLGLeUJOE0pOiAFHcI.jpg,80,500,en,Reservoir Dogs,A botched robbery indicates a police informant...,20.503,/AjTtJNumZyUDz33VtMlF1K8JPsE.jpg,1992-09-02,Reservoir Dogs,False,8.2,10548
1,False,/g6R1OT7ETBLGLeUJOE0pOiAFHcI.jpg,53,500,en,Reservoir Dogs,A botched robbery indicates a police informant...,20.503,/AjTtJNumZyUDz33VtMlF1K8JPsE.jpg,1992-09-02,Reservoir Dogs,False,8.2,10548


In [212]:
#Verify how many movies was extracted from api

#Movie titles extracted from api
mvdb_titles = mvdb_db.groupby('id').first()[['title']].reset_index().rename(columns={'id':'mvdb_id', 'title': 'mvdb_title'})

#Movie titles extracted from api merged with netflix title
titles_mvdb_nflix = mvdb_titles.merge(mv_nflix['nflix_title'], left_on = 'mvdb_title', right_on = 'nflix_title')

#Verify which movies were not extracted from api
nflix_titles_set = set(nflix_db['nflix_title'])
mvdb_titles_set = set(mvdb_titles['mvdb_title'])
missing_titles = nflix_titles_set.difference(mvdb_titles_set)

#Results
#Number of movies which have the same titles
num_movies_nflix = len(nflix_db)
num_movies_same_titles = len(titles_mvdb_nflix)
num_missing_titles = len(missing_titles)

#Display results
print('Result Summary\n')
print('Number of total movies considered in Netflix database:', '\t',f'{num_movies_nflix}')
print('Number of titles extracted which share exactly the same title between both databases:', '\t',f'{num_movies_same_titles}')
print('Number of titles that were not extracted from movie database or did not match exactly:', '\t',f'{num_missing_titles}')
print('Missing titles:')
for i, m_t in enumerate(list(missing_titles)):
    print('\t',i,m_t)


Result Summary

Number of total movies considered in Netflix database: 	 200
Number of titles extracted which share exactly the same title between both databases: 	 173
Number of titles that were not extracted from movie database or did not match exactly: 	 27
Missing titles:
	 0 I
	 1 The School of Rock
	 2 Star Wars: Episode V: The Empire Strikes Back
	 3 X-Men
	 4 Lord of the Rings: The Fellowship of the Ring
	 5 The Butterfly Effect: Director's Cut
	 6 The Shawshank Redemption: Special Edition
	 7 Men in Black
	 8 Lord of the Rings: The Two Towers
	 9 Lord of the Rings: The Return of the King
	 10 Radio
	 11 Dodgeball: A True Underdog Story
	 12 Seven
	 13 Mission: Impossible
	 14 Harry Potter and the Sorcerer's Stone
	 15 Big
	 16 X2: X-Men United
	 17 Collateral
	 18 There's Something About Mary: Special Edition
	 19 Star Wars: Episode II: Attack of the Clones
	 20 Monsters
	 21 Finding Nemo (Widescreen)
	 22 GoodFellas: Special Edition
	 23 Adaptation
	 24 The Matrix: Reloaded
	

In [214]:
titles_mvdb_nflix

,mvdb_id,mvdb_title,nflix_title
0,13,Forrest Gump,Forrest Gump
1,14,American Beauty,American Beauty
2,22,Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse of the Bla...
3,24,Kill Bill: Vol. 1,Kill Bill: Vol. 1
4,38,Eternal Sunshine of the Spotless Mind,Eternal Sunshine of the Spotless Mind
5,70,Million Dollar Baby,Million Dollar Baby
6,77,Memento,Memento
7,85,Raiders of the Lost Ark,Raiders of the Lost Ark
8,87,Indiana Jones and the Temple of Doom,Indiana Jones and the Temple of Doom
9,88,Dirty Dancing,Dirty Dancing


In [219]:
#For each missing movie find equivalent movie database id 
movies_mvdb = {
     'Mission: Impossible' : 954,
     "The Butterfly Effect: Director's Cut" : 1954,
     'Monsters' : 585,
     'Star Wars: Episode V: The Empire Strikes Back' : 1891,
     'Star Wars: Episode II: Attack of the Clones' : 1894,
     "Harry Potter and the Sorcerer's Stone":671,
     'Adaptation':2757,
     'Collateral':1538,
     'Big' : 2280,
     "There's Something About Mary: Special Edition" : 544,
     'X2: X-Men United': 36658,
     'Seven': 807,
     'Finding Nemo (Widescreen)' : 12,
     'Lord of the Rings: The Two Towers':121,
     'GoodFellas: Special Edition':769,
     'The School of Rock': 1584,
     'Shrek (Full-screen)': 808,
     'X-Men': 447399,
     'Lord of the Rings: The Fellowship of the Ring':120,
     'The Shawshank Redemption: Special Edition': 278 ,
     'The Matrix: Reloaded':604,
     'Lord of the Rings: The Return of the King':122,
     'Dodgeball: A True Underdog Story': 9472,
     'I': 2048,
     'Monster': 504,
     'Men in Black': 607,
     'Radio': 13920,
}
len(movies_mvdb)

27

In [220]:
#Create list of all the mvdb ids for each of the movies
mvdb_ids_full_list = list(titles_mvdb_nflix['mvdb_id'])+list(movies_mvdb.values())
len(mvdb_ids_full_list)

200

In [228]:
def extract_data_movie_database_api_details(mvdb_ids):
    '''

    Query details about a movie in TMBD with a movie title id
    
    Input:
    movie_titles : list of movie title ids
    
    Output:
    response_data : Dataframe which holds detailed data on the query
    
    '''

    for i, num in enumerate(mvdb_ids):
        d={}
        url = 'https://api.themoviedb.org/3/movie/'
        movie_num = num
        apikey = f'api_key={KEY}'
        response = requests.get(f'{url}{movie_num}?{apikey}')
        r_json = response.json()
        d['adult'] = [r_json['adult']]
        d['backdrop_path'] = [r_json['backdrop_path']]
        d['genre_ids'] = [r_json['genres']]
        d['id'] = [r_json['id']]
        d['original_language'] = [r_json['original_language']]
        d['original_title'] = [r_json['original_title']]
        d['overview'] = [r_json['overview']]
        d['popularity'] = [r_json['popularity']]
        if r_json.get('belongs_to_collection', None) != None: 
            d['poster_path'] = [r_json['belongs_to_collection']['poster_path']]
        else:
            d['poster_path'] = 0
        d['release_date'] = [r_json['release_date']]
        d['title']= [r_json['title']]
        d['video']= [r_json['video']]
        d['vote_average']= [r_json['vote_average']]
        d['vote_count']= [r_json['vote_count']]

        tmp = pd.DataFrame(d)
        if i == 0: 
            response_data = tmp
        else:
            response_data = pd.concat([response_data, tmp])
            
    return response_data


In [232]:
tmdb_db = extract_data_movie_database_api_details(mvdb_ids_full_list)

In [234]:
tmdb_db = tmdb_db.reset_index(drop = True)
tmdb_db 

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/tlEFuIlaxRPXIYVHXbOSAMCfWqk.jpg,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",13,en,Forrest Gump,A man with a low IQ has accomplished great thi...,36.049,0,1994-07-06,Forrest Gump,False,8.5,20522
1,False,/DztBnZaqmla2sGUW9s8AyOmskT.jpg,"[{'id': 18, 'name': 'Drama'}]",14,en,American Beauty,"Lester Burnham, a depressed suburban father in...",16.495,0,1999-09-15,American Beauty,False,8.0,9260
2,False,/jOh79POQu4hyVIseUxdQxTW7vOf.jpg,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",22,en,Pirates of the Caribbean: The Curse of the Bla...,"Jack Sparrow, a freewheeling 18th-century pira...",48.052,/zRBaZxS5YauLvRYjAdL4AUCwlht.jpg,2003-07-09,Pirates of the Caribbean: The Curse of the Bla...,False,7.7,15942
3,False,/lVy5Zqcty2NfemqKYbVJfdg44rK.jpg,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",24,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",21.700,/tf1nUtw3LJGUGv1EFFi23iz6ngr.jpg,2003-10-10,Kill Bill: Vol. 1,False,8.0,13206
4,False,/ibw3MvF2GLHVcPJd2PDtOQcMDPq.jpg,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",38,en,Eternal Sunshine of the Spotless Mind,"Joel Barish, heartbroken that his girlfriend u...",21.849,0,2004-03-19,Eternal Sunshine of the Spotless Mind,False,8.1,10996
5,False,/oGMomeS7bE43eN8SGJUaKvQnmud.jpg,"[{'id': 18, 'name': 'Drama'}]",70,en,Million Dollar Baby,Despondent over a painful estrangement from hi...,18.993,0,2004-12-15,Million Dollar Baby,False,8.0,7021
6,False,/q2CtXYjp9IlnfBcPktNkBPsuAEO.jpg,"[{'id': 9648, 'name': 'Mystery'}, {'id': 53, '...",77,en,Memento,Leonard Shelby is tracking down the man who ra...,18.517,0,2000-10-11,Memento,False,8.2,10988
7,False,/kCiMExsYuNhYluHxPP2OTmWw7hp.jpg,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",85,en,Raiders of the Lost Ark,When Dr. Indiana Jones – the tweed-suited prof...,24.583,/lpxDrACKJhbbGOlwVMNz5YCj6SI.jpg,1981-06-12,Raiders of the Lost Ark,False,7.9,9083
8,False,/2hMt6zKQsvYvH3ZRe8T6RzAD2XB.jpg,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",87,en,Indiana Jones and the Temple of Doom,"After arriving in India, Indiana Jones is aske...",18.481,/lpxDrACKJhbbGOlwVMNz5YCj6SI.jpg,1984-05-23,Indiana Jones and the Temple of Doom,False,7.3,6650
9,False,/enXtqOyPEVcUStxTChjcBLGQLnv.jpg,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",88,en,Dirty Dancing,Expecting the usual tedium that accompanies a ...,19.140,/gkrOvdbCFuGlxVwCYmczw7aw6Ku.jpg,1987-08-21,Dirty Dancing,False,7.4,4630


In [240]:
#Modify title column to match the titles in netflix
netflix_missing_titles = sorted(missing_titles)
mvdb_missing_titles = ['Adaptation.',
 'Big',
 'Collateral',
 'DodgeBall: A True Underdog Story',
 'Finding Nemo',
 'GoodFellas',
 "Harry Potter and the Philosopher's Stone",
 'I, Robot',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Lord of the Rings: The Return of the King',
 'The Lord of the Rings: The Two Towers',                                    
 'Men in Black',
 'Mission: Impossible',
 'Monster',
 'Monsters, Inc.',
 'Radio',
 'Se7en',
 'Shrek',
 'Star Wars: Episode II - Attack of the Clones',
 'The Empire Strikes Back',
 'The Butterfly Effect',
 'The Matrix Reloaded',
 'School of Rock',
 'The Shawshank Redemption',
 "There's Something About Mary",
 'X-Men: The Mutant Watch',
 'X2']
missing_title_dict = dict(zip(mvdb_missing_titles, netflix_missing_titles ))
missing_title_dict
tmdb_db['title'] = tmdb_db['title'].replace(missing_title_dict)

In [245]:
#cleaning process

def clean_genre_ids(row):
    genres = []
    for genre_id in row['genre_ids']:
        genre = genre_id['name']
        genres.append(genre)
    genres_str = ('_').join(genres)
    return genres_str
        
def clean(df):
    poster_size = 'w300'
    poster_base_url = 'https://image.tmdb.org/t/p/'
    df['backdrop_path'] =  df['backdrop_path'].apply(lambda x : f'{poster_base_url}{poster_size}{x}')
    def make_poster_path(x):
        if x != 0:
            return f'{poster_base_url}{poster_size}{x}'
        else:
            return 0
    df['poster_path'] =  df['poster_path'].apply(lambda x : make_poster_path(x))
    df['release_date'] = pd.to_datetime(df['release_date'])
    return df



In [247]:
tmdb_db['genre_ids'] = tmdb_db.apply(clean_genre_ids, axis = 1)
tmdb_db = clean(tmdb_db)

<h2>The Movie Database API Extracted Data</h2>

In [251]:
tmdb_db 

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,https://image.tmdb.org/t/p/w300https://image.t...,Comedy_Drama_Romance,13,en,Forrest Gump,A man with a low IQ has accomplished great thi...,36.049,0,1994-07-06,Forrest Gump,False,8.5,20522
1,False,https://image.tmdb.org/t/p/w300https://image.t...,Drama,14,en,American Beauty,"Lester Burnham, a depressed suburban father in...",16.495,0,1999-09-15,American Beauty,False,8.0,9260
2,False,https://image.tmdb.org/t/p/w300https://image.t...,Adventure_Fantasy_Action,22,en,Pirates of the Caribbean: The Curse of the Bla...,"Jack Sparrow, a freewheeling 18th-century pira...",48.052,https://image.tmdb.org/t/p/w300https://image.t...,2003-07-09,Pirates of the Caribbean: The Curse of the Bla...,False,7.7,15942
3,False,https://image.tmdb.org/t/p/w300https://image.t...,Action_Crime,24,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",21.700,https://image.tmdb.org/t/p/w300https://image.t...,2003-10-10,Kill Bill: Vol. 1,False,8.0,13206
4,False,https://image.tmdb.org/t/p/w300https://image.t...,Science Fiction_Drama_Romance,38,en,Eternal Sunshine of the Spotless Mind,"Joel Barish, heartbroken that his girlfriend u...",21.849,0,2004-03-19,Eternal Sunshine of the Spotless Mind,False,8.1,10996
5,False,https://image.tmdb.org/t/p/w300https://image.t...,Drama,70,en,Million Dollar Baby,Despondent over a painful estrangement from hi...,18.993,0,2004-12-15,Million Dollar Baby,False,8.0,7021
6,False,https://image.tmdb.org/t/p/w300https://image.t...,Mystery_Thriller,77,en,Memento,Leonard Shelby is tracking down the man who ra...,18.517,0,2000-10-11,Memento,False,8.2,10988
7,False,https://image.tmdb.org/t/p/w300https://image.t...,Adventure_Action,85,en,Raiders of the Lost Ark,When Dr. Indiana Jones – the tweed-suited prof...,24.583,https://image.tmdb.org/t/p/w300https://image.t...,1981-06-12,Raiders of the Lost Ark,False,7.9,9083
8,False,https://image.tmdb.org/t/p/w300https://image.t...,Adventure_Action,87,en,Indiana Jones and the Temple of Doom,"After arriving in India, Indiana Jones is aske...",18.481,https://image.tmdb.org/t/p/w300https://image.t...,1984-05-23,Indiana Jones and the Temple of Doom,False,7.3,6650
9,False,https://image.tmdb.org/t/p/w300https://image.t...,Drama_Music_Romance,88,en,Dirty Dancing,Expecting the usual tedium that accompanies a ...,19.140,https://image.tmdb.org/t/p/w300https://image.t...,1987-08-21,Dirty Dancing,False,7.4,4630


In [249]:
tmdb_db.to_csv('data/mvdb.csv.gz', index = False, compression = 'gzip')

In [250]:
tmdb_db['title'].nunique()

200